<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 오차역전파법(backpropagation)은 가중치 매개변수의 기울기를 효율적으로 계산할 수 있음
- 오차역전파란 풀어서 설명하면 '오차를 역방향으로 전파하는 방법'이라고 할 수 있음




## 계산그래프(computational graph)
- 계산그래프를 통해서 오차역전파에 대해 시각적으로 이해할 수 있음
- 계산그래프란 계산 과정을 그래프로 나타낸 것

### 계산 그래프로 풀어보기
- 슈퍼에서 1개에 100원인 사과 2개를 구매했을 때, 지불 금액은 얼마인가(단, 소비세가 10% 부과됨)
- 슈퍼에서 한개에 100원인 사과 2개, 1개에 150원인 귤 3개를 구매했을 때, 지불 금액은 얼마인가(단, 소비세는 10% 부과됨)

- 계산 그래프를 푸는 흐름
  - 계산 그래프를 구성한 후 왼쪽에서 오른쪽으로 계산 진행(순전파)
  
  * 순전파 : 계산 그래프의 출발점에서 종착점으로의 전파
  * 역전파 : 계산 그래프의 종착점에서 출발점으로 전파

- 역전파는 미분을 계산할 때 중요한 역할을 수행함 

### 국소적 계산
- 계산 그래프의 특징 : 국소적 계산을 전파하여 최종 결과를 얻음

* 국소적 : 자신과 직접 관계된 작은 범위

  - 각 노드는 자신과 관련된 계산 외에는 관심x

- 계산 그래프의 이점 
  - 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문지를 단순화 가능 + 중간 계산 결과를 모두 보관할 수 있음
  - 가장 중요한 부분은 역전파를 통해 미분을 효율적으로 계산 가능
  (국소적 미분을 수행한 후 그 값을 왼쪽으로 전달)

## 단순한 계층 구현하기

In [30]:
# 곱셈 계층 구현하기
class MulLayer :
  def __init__ (self) :
    # 순전파시 입력 값을 유지하기 위해서 사용
    self.x = None
    self.y = None

  def forward(self, x, y) :
    self.x = x
    self.y = y
    out = x * y

    return out
  
  def backward(self, dout) :
    dx = dout * self.y # x와 y를 바꾼다
    dy = dout * self.x 

    return dx, dy

In [31]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [32]:
# 역전파를 통해  각 변수에 대한 미분 값 구하기
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_price, dapple_num, dtax)

1.1 110.00000000000001 200


In [33]:
# 덧셈 계층 구현하기
class AddLayer :
  def __init__(self) :
    pass

  def forward(self, x, y) :
    out = x+y
    return out
  
  def backward(self, dout) :
    dx = dout * 1
    dy = dout * 1
    return dx, dy 

In [34]:
# 위에서 구현한 덧셈 계층과 곱셈 계층을 통해 사과 2개와 귤 3개를 사는 상황을 구현
apple = 100
apple_num = 2 
orange = 15
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice  = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

269.5
110.00000000000001 2.2 3.3000000000000003 16.5 245


## 활성화 함수 계층 구현
### ReLU 계층
- 순전파 때 입력 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보내고,
순전파 때 입력 x가 0보다 작거나 같으면 역전파는 그 값을 하류로 신호 보내지 않음(0을 보냄)



In [42]:
## ReLu 계층 구현
class Relu :
  def __init__(self) :
    self.mask = None

  def forward(self, x) :
    self.mask = ( x <= 0)
    out = x.copy()
    out[self.mask] = 0
    return out
    
  def backward(self, dout) :
    dout[self.mask] = 0
    dx = dout
    return dx

In [ ]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0 )
print(mask)



[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### Sigmoid 계층
- 순전파의 출력 y만으로도 역전파를 계산할 수 있음

In [ ]:
class Sigmoid :
  def __init__(self) : 
    self.out = None
  
  def forward(self, x ) :
    out = 1 / (1+np.exp(-x))
    self.out = out # 순전파의 출력을 보관 후 역전파 계산 시 사용
    return out
  
  def backward(self, dout) :
    dx = dout * (1.0 - self.out) * self.out
    return dx

## Affine/Softmax 계층 구현


In [56]:
class Affine :
  def __init__(self, W, b) :
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x) :
    self.x = x
    # print(x)
    out = np.dot(x, self.W) + self.b

    return out
  
  def backward(self, dout) :
    # print(dout)
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis = 0)

    return dx
  

In [35]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [44]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### Softmax-with-Loss 계층
- 출력층에서 사용하는 소프트 맥스 함수(입력 값을 정규화(출력의 합이 1이됨)하여 출력)
- 신경망 추론 시에는 softmax 계층이 필요없으나 신경망을 학습할 때에는 Softmax 계층이 필요
- 소프트맥스 계층 + 손실 함수인 교차 엔트로피 오차도 포함한 계층을 구현해봄
- 소프트맥스의 역전파는 소프트맥스의 출력값과 정답 레이블의 차분(오차)가 앞 계층에 전해지는 것
- 회귀 출력층에서 사용하는 '항등함수'의 손실 함수로 '오차제곱합'을 이용하는 이유는 이를 사용하면 역전파의 결과가 출력값-정답인 차분으로 말끔히 떨어지기 때문

In [53]:
class SoftmaxWithLoss :
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답레이블(원-핫 벡터)

  def forward(self, x, t) :
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout = 1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size # 역전파 시 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층에 전파함
    return dx

## 오차역전파법 구현
- 그동안 했던 것들을 조합하여 오차역전파 구현
- 신경망 학습 전체 그림
  - 전제 : 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 학습 과정 존재
  - 1. 미니배치 : 훈련 데이터 중 일부를 무작위 추출한 미니배치의 손실 함수 값을 줄이는 것이 목표
  - 2. 기울기 산출 : 미니배치릐 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함
  - 3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
  - 4. 1~3 반복

In [17]:
# 오차역전파법을 적용한 신경망은 수치 미분으로 구현한 것보다 효율적으로 기울기 산출 가능

In [54]:
import numpy as np
from collections import OrderedDict # 순서가 보장되는 dict

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

class TwoLayerNet :
  def __init__(self, input_size, hidden_size, output_size,
               weight_init_std = 0.01) :
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()
  
  def predict(self, x) :
    for layer in self.layers.values() :
      # print(layer)
      x = layer.forward(x)
    
    return x

  def loss(self, x, t) :
    y = self.predict(x)
    return self.lastLayer.forward(y, t)
  
  def accuracy(self, x, t) :
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t) :
    loss_W = lambda W : self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W , self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W , self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W , self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W , self.params['b2'])

    return grads            
  
  def gradient(self, x, t) :
    # 순전파
    self.loss(x, t)

    # 역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values()) # OrderedDict 객체이므로 입력한 순서가 저장되어 있음. 따라서 역전파 시에는
                                        # 입력한 순서의 반대로 계층을 호출하면 처리가 됨
    layers.reverse()
    for layer in layers :
      dout = layer.backward(dout)
    
    # 결과 저장
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db

    return grads


### 오차역전파법으로 구한 기울기 검증
- 기울기를 구하기 위해 수치미분을 사용하는 경우, 구현이 쉽지만 오차역전파를 사용하는 경우에는 구현 시 종종 실수가 발생할 수 있음
  - 따라서, 수치 미분으로 구한 결과와 오차역전파로 구한 결과를 비교하여 검증.
  이러한 것을 '기울기 확인(gradient check)'라고 함
  - 두 가지 경우의 차가 0이 될 확률은 적음(컴퓨터가 할 수 있는 계산의 정밀도가 유한). 하지만 올바르게 구현되었다면 0에 아주 가까운 값이 될 것이므로 오차가 클 경우, 오차역전파법을 잘못 구현한 것인지 확인 필요

In [ ]:
# 각 가중치 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 구하는 방식을 통해 검증 가능
# 예시 코드
for key in grad_numerical_keys() :
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))

### 오차역전파법을 사용한 학습 구현

In [22]:
# 데이터 불러오기
try:
    import urllib.request
except ImportError:
    raise ImportError('You should use Python 3.x')
import os.path
import gzip
import pickle
import os
import numpy as np
import sys

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

__file__ = os.pardir
# print(__file__)
dataset_dir = os.path.dirname(os.path.abspath(__file__))
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784


def _download(file_name):
    file_path = dataset_dir + "/" + file_name
    
    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")
    
def download_mnist():
    for v in key_file.values():
       _download(v)
        
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")
    
    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")    
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")
    
    return data
    
def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])    
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])
    
    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T
    

def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """MNIST 데이터셋 읽기
    
    Parameters
    ----------
    normalize : 이미지의 픽셀 값을 0.0~1.0 사이의 값으로 정규화할지 정한다.
    one_hot_label : 
        one_hot_label이 True면、레이블을 원-핫(one-hot) 배열로 돌려준다.
        one-hot 배열은 예를 들어 [0,0,1,0,0,0,0,0,0,0]처럼 한 원소만 1인 배열이다.
    flatten : 입력 이미지를 1차원 배열로 만들지를 정한다. 
    
    Returns
    -------
    (훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블)
    """
    if not os.path.exists(save_file):
        init_mnist()
        
    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)
    
    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0
            
    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])    
    
    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label']) 


# if __name__ == '__main__':
#     init_mnist()

# MNIST 데이터셋을 내려받아 그 이미지를 넘파이 배열로 변화해주는 파이썬 스크립트 사용
# import sys, os
# sys.path.append(os.pardir)
# # from dataset.mnist import load_mnist
# # load_mnist()

# (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# print(x_train.shape)
# print(t_train.shape)

In [59]:
from tqdm import tqdm
import numpy as np

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# print(x_train.shape)
# print(t_train.shape)

train_loss_list = []
train_acc_list = []
test_acc_list = []




# 하이퍼파라미터 : 사람이 조정하는 매개변수
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

# 1에포크 당 반복 수 
iter_per_epoch = max(train_size/batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in tqdm(range(iters_num)) :
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  # grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch ) # 성능 개선판(역전파)

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2') :
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  # 1에포크당 정확도 계산
  if i % iter_per_epoch == 0 :
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print('train acc, test acc | '+str(train_acc)+","+str(test_acc))

  0%|          | 32/10000 [00:00<43:13,  3.84it/s] 

train acc, test acc | 0.15275,0.1617


  6%|▋         | 632/10000 [00:02<00:56, 166.06it/s]

train acc, test acc | 0.9062833333333333,0.9092


 13%|█▎        | 1255/10000 [00:05<00:51, 169.57it/s]

...

 91%|█████████ | 9057/10000 [00:36<00:05, 173.77it/s]

train acc, test acc | 0.9772833333333333,0.9692


 96%|█████████▋| 9642/10000 [00:38<00:02, 169.33it/s]

train acc, test acc | 0.9793333333333333,0.9713


100%|██████████| 10000/10000 [00:39<00:00, 252.23it/s]
